# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import google_maps_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = os.path.join("..", "WeatherPy", "cities.csv")
weather_data_df = pd.read_csv(weather_data)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=google_maps_key)

In [4]:
humidity = weather_data_df["Humidity"].astype(float)
humidity.head()

0    62.0
1    99.0
2    93.0
3    58.0
4    60.0
Name: Humidity, dtype: float64

In [5]:
locations = weather_data_df[["Latitude", "Longitude"]].astype(float)
locations.head()

,Latitude,Longitude
0,-42.78,147.06
1,34.68,112.45
2,66.04,-17.34
3,37.10,-113.58
4,-38.00,-57.56


In [6]:
weather_data_df = weather_data_df.dropna()
weather_data_df.head()

,City,Country,Date,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
0,new norfolk,AU,1586627299,52.00,62,90,10.29,-42.78,147.06
1,luoyang,CN,1586627704,53.01,99,0,4.09,34.68,112.45
2,husavik,IS,1586627704,37.40,93,75,18.34,66.04,-17.34
3,saint george,US,1586627283,61.00,58,1,4.88,37.10,-113.58
4,mar del plata,AR,1586627146,73.00,60,40,14.99,-38.00,-57.56


In [7]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather = weather_data_df.loc[(weather_data_df["Temp"]>=75) & (weather_data_df["Temp"]<=90) \
                                    & (weather_data_df["Wind Speed"]<15) \
                                    & (weather_data_df["Cloudiness"]<3)].dropna()
ideal_weather

,City,Country,Date,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude
55,cockburn town,TC,1586627571,79.41,71,0,4.47,21.46,-71.14
60,new port richey,US,1586627722,86.00,51,1,3.31,28.24,-82.72
74,athmallik,IN,1586627727,82.83,22,0,1.83,20.72,84.53
91,gainesville,US,1586627672,82.00,36,1,9.17,29.65,-82.32
126,dingle,PH,1586627744,77.40,73,2,5.68,11.00,122.67
147,pisco,PE,1586627311,75.00,30,1,10.29,-13.70,-76.22
192,jaora,IN,1586627767,85.46,14,2,5.97,23.63,75.13
217,ojinaga,MX,1586627321,78.80,10,1,6.93,29.57,-104.42
272,kavaratti,IN,1586627311,85.01,68,1,4.03,10.57,72.64
274,kankon,IN,1586627795,78.85,87,0,3.91,15.02,74.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = ideal_weather
hotel_name = []

params = {
    "radius": 5000,
    "types": "lodging",
    "key": google_maps_key
}

for index, row in ideal_weather.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except IndexError:
        print(f"No hotel")

No hotel


In [14]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Date,Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Hotel Name
55,cockburn town,TC,1586627571,79.41,71,0,4.47,21.46,-71.14,Osprey Beach Hotel
60,new port richey,US,1586627722,86.00,51,1,3.31,28.24,-82.72,Fairfield Inn & Suites by Marriott Holiday Tar...
74,athmallik,IN,1586627727,82.83,22,0,1.83,20.72,84.53,Swadesh motors ..
91,gainesville,US,1586627672,82.00,36,1,9.17,29.65,-82.32,Wyndham Garden Gainesville
126,dingle,PH,1586627744,77.40,73,2,5.68,11.00,122.67,Camp Pasica
147,pisco,PE,1586627311,75.00,30,1,10.29,-13.70,-76.22,Hostal San Isidro Oficial
192,jaora,IN,1586627767,85.46,14,2,5.97,23.63,75.13,BLM PALACE
217,ojinaga,MX,1586627321,78.80,10,1,6.93,29.57,-104.42,Suites Las Palmas
272,kavaratti,IN,1586627311,85.01,68,1,4.03,10.57,72.64,Bismillah Hotel And Lodging
274,kankon,IN,1586627795,78.85,87,0,3.91,15.02,74.02,The Gardenia Resort


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…